In [2]:
import torch
import torchvision
import gym

import random
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.autograd as autograd
import torch.nn.functional as F
import gym
from gym.envs.registration import register
register(
   id='FrozenLakeNotSlippery-v0',
   entry_point='gym.envs.toy_text:FrozenLakeEnv',
   kwargs={'map_name' : '4x4', 'is_slippery': False},
   max_episode_steps=100,
   reward_threshold=0.78, # optimum = .8196
)

#env = gym.make('FrozenLake8x8-v0')
#env = gym.make('FrozenLake-v0')
env = gym.make('FrozenLakeNotSlippery-v0')
env.render()
class pi_net(nn.Module):
    def __init__(self):
        super(pi_net, self).__init__()
        self.linear1 = nn.Linear(1, 20)
        self.linear2 = nn.Linear(20, 4 + 1)
        

    def forward(self, x):
        #print(x.shape)
        #print(x)
        x = F.relu(self.linear1(x))
        #x = F.softmax(self.linear2(x), dim=0)
        x = self.linear2(x)
        x = x.view(-1,5)
        #print(x.shape)
        #print(x)
        actions = x[:,:4]
        value = x[:,4]
        return actions, value
    
    


SFFF
FHFH
FFFH
HFFG


In [4]:
# custom weights initialization 
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        #m.weight.data.normal_(0.0, 0.02)
        #m.weight.data.uniform_(0.0, 0.02)
        m.weight.data.fill_(0.5)

import gym
import numpy as np
import torch.optim as optim
from torch.distributions import Categorical
mse = nn.MSELoss()

NUM_EPISODES = 1000000
GAMMA = 0.9
net = pi_net()
net.apply(weights_init)
optimizer = optim.RMSprop(net.parameters(), lr=0.0001)


score = []
times_trained = 0
times_reach_goal = 0
for k in range(NUM_EPISODES):
    done = False
    observation = env.reset()
    #observation, reward, done, info = env.step(env.action_space.sample()) # take a random action
    episode_series = []
    while not done:
        # Get action from pi
        # action = env.action_space.sample()
        np_observation = np.array(observation)
        #np_observation = np.expand_dims(np_observation, axis=0)
        np_observation = np.expand_dims(np_observation, axis=0)
        observation_tensor = torch.FloatTensor(np_observation) 
        #print(observation_tensor)
        #net.eval()
        #print("before eval")
        action_probs, value = net(observation_tensor)
        action_probs_orig = action_probs
        #print("action_probs after net")
        #print(action_probs)
        #FOR EXPLORATION: 
        #action_probs = F.dropout(action_probs, p=0.3, training=True)
        #print("action_probs after dropout")
        #print(action_probs)
        action_probs = F.softmax(action_probs, dim=1)
        #print("action_probs after softmax")
        #print(action_probs)
        #action = action_probs.multinomial(num_samples=1)
        m = Categorical(action_probs)
        action = m.sample()
        
        
        
        #print("after eval")
        #print("action_probs")
        #print(action_probs)
        log_prob = m.log_prob(action)
        #print("log_prob")
        #print(log_prob)
        #break
        #print("softmax")
        #print(action_probs)
        #print("action")
        #print(str(action.item()))
        #print(type(prob.multinomial))
        
        #break
        # Execute action in environment.
        
        if k%10000 == 0:
            print("action_probs_orig ")
            print(action_probs_orig)
            print("On state="+ str(observation) + ", selected action=" + str(action.item()) + " , ")
        
        observation, reward, done, info = env.step(action.item()) 
        
        if k%10000 == 0:
            print("new state="+ str(observation) + ", done="+str(done))
        #if done and reward != 1.0:
        #    reward = -1.0
        step_data = [observation, action,log_prob, value, reward, done, info]
        episode_series.append(step_data)
        #env.render()
        
   
    if len(score) < 100:
        score.append(reward)
    else:
        score[k % 100] = reward

    if k%1000 == 0:
        print("Episode {} finished after {} timesteps with r={}. Running score: {}. Times trained: {}. Times reached goal: {}.".format(k, len(episode_series), reward, np.mean(score), times_trained, times_reach_goal))
        times_trained = 0
        times_reach_goal = 0
        #print("Game finished. " + "-" * 5)
        #print(len(episode_series))
#         for param in net.parameters():
#             print(param.data)
        

    
    discounted_rewards = []
    actions = []
    values = []
    log_probs = []
    
    R = 0
    for i in reversed(range(len(episode_series))): # Build the different containers: action, value, reward
        [observation, action, log_prob, value, reward, done, info] = episode_series[i]
        R = reward + R * GAMMA
        discounted_rewards.append(R)
        actions.append(action)
        values.append(value)
        log_probs.append(log_prob)
        
    value_loss = 0
    policy_log_loss = 0
    for action, value, log_prob, reward in zip(actions, values, log_probs, discounted_rewards):
        reward_diff = reward - value.item() # Treat critic value as baseline
        #action.reinforce(reward_diff) # Try to perform better than baseline
        policy_log_loss += -log_prob * reward_diff
        value_loss += mse(value, torch.Tensor([reward])) # Compare with actual reward
        
    
    
    
    if R > 0.0 or True: # Optimize only if rewards are non zero.
        #print "Reward list"
        #print rewards_list
        optimizer.zero_grad()
        nodes = [value_loss] + [policy_log_loss]
        gradients = [torch.ones(1)] + [torch.ones(1)] # No gradients for reinforced values
        autograd.backward(nodes, gradients)
        optimizer.step()
        times_trained = times_trained + 1
    
    if R > 0.0:
        times_reach_goal = times_reach_goal + 1
        
    
    

action_probs_orig 
tensor([[ 2.9848,  2.8283,  3.1146,  3.1600]])
On state=0, selected action=0 , 
new state=0, done=False
action_probs_orig 
tensor([[ 2.9848,  2.8283,  3.1146,  3.1600]])
On state=0, selected action=0 , 
new state=0, done=False
action_probs_orig 
tensor([[ 2.9848,  2.8283,  3.1146,  3.1600]])
On state=0, selected action=2 , 
new state=1, done=False
action_probs_orig 
tensor([[ 6.1218,  5.9652,  6.2515,  6.2969]])
On state=1, selected action=0 , 
new state=0, done=False
action_probs_orig 
tensor([[ 2.9848,  2.8283,  3.1146,  3.1600]])
On state=0, selected action=1 , 
new state=4, done=False
action_probs_orig 
tensor([[ 20.6047,  20.4481,  20.7344,  20.7798]])
On state=4, selected action=3 , 
new state=0, done=False
action_probs_orig 
tensor([[ 2.9848,  2.8283,  3.1146,  3.1600]])
On state=0, selected action=0 , 
new state=0, done=False
action_probs_orig 
tensor([[ 2.9848,  2.8283,  3.1146,  3.1600]])
On state=0, selected action=2 , 
new state=1, done=False
action_probs

KeyboardInterrupt: 